In [1]:
import pandas as pd
import json
import os
import cv2
from PIL import ImageDraw
from PIL import Image
import csv 

json_file_path="/opt/ml/input/Project/bug_data/Training/label"
file_list = os.listdir(json_file_path)
import pandas as pd
df = pd.DataFrame(file_list)      
df.transpose()
df.to_csv("./annotation_list.csv", mode='w')

In [2]:
import pandas as pd
import numpy as np
import json

class_to_label = {
        0 : 0,
        10 : 1,
        11 : 2,
        12 : 3,
        15 : 4
}


root = {'info': {'year': 2021,
  'version': '1.0',
  'description': 'Bug Detection',
  'contributor': 'aihub',
  'url': None,
  'date_created': None},
 'licenses': [{'id': 0,
   'name': None,
   'url': None}],
 'categories':[{'id': 0,
   'name': 'Normal',
   'supercategory': 'Bug'},
              {'id': 1,
   'name': 'xylostella',
   'supercategory': 'Bug'},
              {'id': 2,
   'name': 'white butterfly',
   'supercategory': 'Bug'},
              {'id': 3,
   'name': 'striolata',
   'supercategory': 'Bug'},
              {'id': 4,
   'name': 'Pentatomidae',
   'supercategory': 'Bug'}],
        'images':[],
       'annotations':[]}
image_list = pd.read_csv("./annotation_list.csv")
data_infos = []
img_annos= []
id=0
ann_id = 0
for file_name in image_list['0']:
    with open("/opt/ml/input/Project/bug_data/Training/label/"+file_name,'r') as j: #annotation 경로 추가
        dic = json.loads(j.read())
        date_captured = dic['description']['date']
        width = dic['description']['width']
        height = dic['description']['height']
        img_name = dic['description']['image']
        img_info = {'width': 512,
                       'height': 512,
                       'file_name': img_name,
                       'license': 0,
                       'flickr_url': None,
                       'coco_url': None,
                       'date_captured': date_captured,
                       'id': id}
        #print(dic)
        data_infos.append(img_info)
        
        
        object_list = dic['annotations']['object']
        
        x_scale = 512 /  width
        y_scale = 512 / height
        for object_one in object_list:
            xtl = int(object_one['points'][0]['xtl'] * x_scale)
            ytl = int(object_one['points'][0]['ytl'] * y_scale)
            b_width = int((object_one['points'][0]['xbr']-object_one['points'][0]['xtl'])*x_scale)
            b_height = int((object_one['points'][0]['ybr']-object_one['points'][0]['ytl'])*y_scale)
            
           
            
            
            
            label =class_to_label[object_one['class']]
            bboxes = [xtl,ytl,b_width,b_height]
        
        
        
            img_anno = {
                'image_id':id,
                'category_id':label,
                'area':512*512,
                'bbox':bboxes,
                'iscrowd':0,
                'id':ann_id
            }
            ann_id+=1
            img_annos.append(img_anno)
        id+=1
        
root['images']=data_infos
root['annotations']=img_annos

#저장
with open("./val_coco_annotations.json",'w') as f:
    json.dump(root, f, ensure_ascii=False, indent=4)
    
print("finish")

finish
